In [3]:
import os
os.chdir("../")

In [4]:
os.getcwd()

'/home/izam/coding/Legal-Advisor-LLM'

In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [14]:
PINECONE_API_KEY = input("API key: ")
PINECONE_API_ENV = "gcp-starter"

In [15]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [16]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [17]:
extracted_data = load_pdf("data/")

In [18]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [21]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 2039


In [19]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

In [21]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [24]:
from langchain_pinecone import PineconeVectorStore

index_name="llm-chatbot"

In [ ]:
#Creating Embeddings for Each of The Text Chunks & storing

docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [25]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What to do when there is a curruption"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='article; or  \n (b) that for the proper administration of the National C apital \nTerritory it is necessary or expedient so to do,  \nthe President may by order suspend the operation of any provision of article \n239AA or of all or any of the provisions of any law made in pursuance of that \narticle for such period and subject to such c onditions as may be specified in \nsuch law and make such incidental and consequential provisions as may appear'), Document(page_content='discharge of the normal duties of the State if such dire ction had not been given, \nthere shall be paid by the Government of India to the State such sum as may be \nagreed, or, in default of agreement, as may be determined by an arbitrator \nappointed by the Chief Justice of India, in respect of the extra costs so in curred \nby the State.'), Document(page_content='unless at least fourteen days’ notice has been given of the intention to move the \nresolution:  \nProvided further that, w

In [26]:
prompt_template="""
Use the following pieces of information about legal terms or constitution to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [27]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

In [28]:
llm = CTransformers(model="/home/izam/coding/Legal-Advisor-LLM/saved_models/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",streaming=True, 
                    config={'max_new_tokens':256,'temperature':0.6,'context_length':-1})

In [29]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [30]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

/home/izam/anaconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  The term "law" in Article 372 of the Indian Constitution refers to any Ordinance, order, bye-law, rule, regulation, notification, custom or usage having the force of law in India. This includes laws passed or made by a Legislature or other competent authority in India before the commencement of the Constitution and not previously repealed, as well as any adaptations and modifications that may be made to such laws under Article 372.
